In [0]:
print("Starting Gold Layer with Azure Storage...")

from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime

In [0]:
# Get task values from Silver layer
# Define the STORAGE_ACCOUNT variable
STORAGE_ACCOUNT = None
try:
    silver_customers_count = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="silver_customers_count", debugValue=500)
    silver_orders_count = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="silver_orders_count", debugValue=1000)
    silver_quality_score = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="silver_quality_score", debugValue=100.0)
    silver_revenue = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="silver_revenue", debugValue=250000.0)
    revenue_variance = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="revenue_variance_pct", debugValue=0.0)
    data_retention_pct = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="data_retention_percentage", debugValue=100.0)
    STORAGE_ACCOUNT = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="STORAGE_ACCOUNT", debugValue="dataworks")
    
    # Get original data generation metrics
    original_total_revenue = dbutils.jobs.taskValues.get(taskKey="data_generation", key="total_revenue", debugValue=250000.0)
    original_avg_order_value = dbutils.jobs.taskValues.get(taskKey="data_generation", key="avg_order_value", debugValue=250.0)
    original_active_customers = dbutils.jobs.taskValues.get(taskKey="data_generation", key="active_customers", debugValue=425)
    
    print(f"📋 Pipeline metrics received:")
    print(f"   Silver customers: {silver_customers_count}")
    print(f"   Silver orders: {silver_orders_count}")
    print(f"   Silver quality: {silver_quality_score}%")
    print(f"   Revenue: ${silver_revenue:,.2f}")
    print(f"   Data retention: {data_retention_pct:.1f}%")
    
except Exception as e:
    print(f"⚠️ Could not retrieve Silver task values (running standalone): {e}")
    silver_quality_score = 100.0
    silver_revenue = 250000.0

# Authentication - Replace with your credentials
spark.conf.set(f"fs.azure.account.key.{STORAGE_ACCOUNT}.dfs.core.windows.net", "access-key")

# Azure Storage Path Configuration
GOLD_PATH = f'abfss://gold@{STORAGE_ACCOUNT}.dfs.core.windows.net/delta/'

# Create gold directory
dbutils.fs.mkdirs(GOLD_PATH)

# Database setup
spark.sql("CREATE DATABASE IF NOT EXISTS ecommerce_gold")
spark.sql("USE ecommerce_gold")

print(f"✅ Gold layer configured: {GOLD_PATH}")

In [0]:
# Load Silver tables
silver_customers = spark.table("ecommerce_silver.customers")
silver_products = spark.table("ecommerce_silver.products")
silver_orders = spark.table("ecommerce_silver.orders")
silver_order_items = spark.table("ecommerce_silver.order_items")
customer_summary = spark.table("ecommerce_silver.customer_summary")

print("✅ Silver tables loaded successfully")

In [0]:
# Sales Dashboard Analytics
print("🔄 Creating sales dashboard metrics...")

sales_dashboard = (silver_orders
    .groupBy("order_year", "order_month", "status")
    .agg(
        F.count("order_id").alias("order_count"),
        F.sum("total_amount").alias("revenue"),
        F.avg("total_amount").alias("avg_order_value"),
        F.countDistinct("customer_id").alias("unique_customers"),
        F.sum("shipping_cost").alias("total_shipping"),
        F.sum("tax_amount").alias("total_tax"),
        F.min("order_date").alias("period_start"),
        F.max("order_date").alias("period_end")
    )
    .withColumn("month_year", F.concat(F.col("order_year"), F.lit("-"), 
                                     F.lpad(F.col("order_month"), 2, "0")))
    .withColumn("revenue_per_customer", F.round(F.col("revenue") / F.col("unique_customers"), 2))
    .withColumn("shipping_rate_pct", F.round(F.col("total_shipping") / F.col("revenue") * 100, 2))
)

# Save to Gold container
sales_dashboard_path = f"{GOLD_PATH}sales_dashboard"
sales_dashboard.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("order_year") \
    .option("mergeSchema", "true") \
    .option("path", sales_dashboard_path) \
    .saveAsTable("ecommerce_gold.sales_dashboard")

sales_dashboard_count = sales_dashboard.count()
print(f"✅ Sales dashboard: {sales_dashboard_count} records")

In [0]:
# Customer Segmentation Analytics
print("🔄 Creating customer segmentation analysis...")

customer_segments = (customer_summary
    .withColumn("customer_tier",
        F.when(F.col("total_spent") >= 1000, "VIP")
        .when(F.col("total_spent") >= 500, "Premium") 
        .when(F.col("total_spent") >= 200, "Standard")
        .otherwise("Basic"))
    .withColumn("recency_status",
        F.when(F.col("days_since_last_order") <= 30, "Active")
        .when(F.col("days_since_last_order") <= 90, "Declining")
        .otherwise("Inactive"))
    .withColumn("frequency_tier",
        F.when(F.col("total_orders") >= 10, "High")
        .when(F.col("total_orders") >= 5, "Medium")
        .otherwise("Low"))
    .withColumn("monetary_tier",
        F.when(F.col("total_spent") >= 1000, "High")
        .when(F.col("total_spent") >= 300, "Medium")
        .otherwise("Low"))
)

segment_summary = (customer_segments
    .groupBy("customer_tier", "recency_status", "segment", "country")
    .agg(
        F.count("customer_id").alias("customer_count"),
        F.sum("total_spent").alias("total_revenue"),
        F.avg("total_spent").alias("avg_customer_value"),
        F.avg("total_orders").alias("avg_orders_per_customer"),
        F.avg("avg_order_value").alias("avg_order_size"),
        F.avg("order_frequency").alias("avg_order_frequency")
    )
    .withColumn("revenue_per_customer", F.round(F.col("total_revenue") / F.col("customer_count"), 2))
    .withColumn("segment_performance_score", 
        F.round((F.col("avg_customer_value") / 100) + (F.col("avg_order_frequency") * 10), 1))
)

# Save to Gold container
customer_segments_path = f"{GOLD_PATH}customer_segments"
segment_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .option("path", customer_segments_path) \
    .saveAsTable("ecommerce_gold.customer_segments")

customer_segments_count = segment_summary.count()
print(f"✅ Customer segments: {customer_segments_count} records")

In [0]:
# Product Performance Analytics
print("🔄 Creating product performance analysis...")

product_performance = (silver_order_items
    .groupBy("category", "price_category")
    .agg(
        F.sum("line_total").alias("category_revenue"),
        F.sum("profit").alias("category_profit"),
        F.sum("quantity").alias("units_sold"),
        F.countDistinct("order_id").alias("orders_with_category"),
        F.countDistinct("product_id").alias("unique_products"),
        F.avg("unit_price").alias("avg_selling_price"),
        F.avg("discount_percent").alias("avg_discount_percent"),
        F.sum("discount_amount").alias("total_discounts_given")
    )
    .withColumn("revenue_per_order", F.round(F.col("category_revenue") / F.col("orders_with_category"), 2))
    .withColumn("profit_margin_pct", F.round(F.col("category_profit") / F.col("category_revenue") * 100, 2))
    .withColumn("avg_units_per_order", F.round(F.col("units_sold") / F.col("orders_with_category"), 2))
    .withColumn("revenue_per_product", F.round(F.col("category_revenue") / F.col("unique_products"), 2))
    .withColumn("discount_impact_pct", F.round(F.col("total_discounts_given") / F.col("category_revenue") * 100, 2))
)

# Save to Gold container
product_performance_path = f"{GOLD_PATH}product_performance"
product_performance.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .option("path", product_performance_path) \
    .saveAsTable("ecommerce_gold.product_performance")

product_performance_count = product_performance.count()
print(f"✅ Product performance: {product_performance_count} records")

In [0]:
# Executive KPIs
print("🔄 Creating executive KPIs...")

executive_kpis = spark.sql("""
SELECT 'Overall' as period,
    COUNT(DISTINCT order_id) as total_orders,
    ROUND(SUM(total_amount), 2) as total_revenue,
    COUNT(DISTINCT customer_id) as active_customers,
    ROUND(AVG(total_amount), 2) as avg_order_value,
    ROUND(SUM(total_amount) / COUNT(DISTINCT customer_id), 2) as revenue_per_customer,
    COUNT(DISTINCT CASE WHEN status = 'Delivered' THEN order_id END) as completed_orders,
    ROUND(COUNT(DISTINCT CASE WHEN status = 'Delivered' THEN order_id END) * 100.0 / COUNT(DISTINCT order_id), 2) as completion_rate_pct,
    ROUND(SUM(shipping_cost), 2) as total_shipping_costs,
    ROUND(SUM(tax_amount), 2) as total_tax_collected
FROM ecommerce_silver.orders
UNION ALL
SELECT 'Current Month' as period,
    COUNT(DISTINCT order_id) as total_orders,
    ROUND(SUM(total_amount), 2) as total_revenue,
    COUNT(DISTINCT customer_id) as active_customers,
    ROUND(AVG(total_amount), 2) as avg_order_value,
    ROUND(SUM(total_amount) / COUNT(DISTINCT customer_id), 2) as revenue_per_customer,
    COUNT(DISTINCT CASE WHEN status = 'Delivered' THEN order_id END) as completed_orders,
    ROUND(COUNT(DISTINCT CASE WHEN status = 'Delivered' THEN order_id END) * 100.0 / COUNT(DISTINCT order_id), 2) as completion_rate_pct,
    ROUND(SUM(shipping_cost), 2) as total_shipping_costs,
    ROUND(SUM(tax_amount), 2) as total_tax_collected
FROM ecommerce_silver.orders 
WHERE YEAR(order_date) = YEAR(current_date()) 
    AND MONTH(order_date) = MONTH(current_date())
""")

# Save to Gold container
executive_kpis_path = f"{GOLD_PATH}executive_kpis"
executive_kpis.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .option("path", executive_kpis_path) \
    .saveAsTable("ecommerce_gold.executive_kpis")

executive_kpis_count = executive_kpis.count()
print(f"✅ Executive KPIs: {executive_kpis_count} records")

In [0]:
# Monthly Trends Analysis
print("🔄 Creating monthly trends analysis...")

monthly_trends = (silver_orders
    .filter(F.col("status") == "Delivered")
    .groupBy("order_year", "order_month")
    .agg(
        F.count("order_id").alias("orders"),
        F.sum("total_amount").alias("revenue"),
        F.countDistinct("customer_id").alias("customers"),
        F.avg("total_amount").alias("avg_order_value"),
        F.sum("shipping_cost").alias("shipping_costs"),
        F.sum("tax_amount").alias("tax_collected"),
        F.countDistinct(F.when(F.col("is_weekend"), F.col("order_id"))).alias("weekend_orders")
    )
    .withColumn("month_year", F.concat(F.col("order_year"), F.lit("-"), 
                                     F.lpad(F.col("order_month"), 2, "0")))
    .withColumn("revenue_per_customer", F.round(F.col("revenue") / F.col("customers"), 2))
    .withColumn("weekend_order_pct", F.round(F.col("weekend_orders") * 100.0 / F.col("orders"), 2))
    .orderBy("order_year", "order_month")
)

# Save to Gold container
monthly_trends_path = f"{GOLD_PATH}monthly_trends"
monthly_trends.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .option("path", monthly_trends_path) \
    .saveAsTable("ecommerce_gold.monthly_trends")

monthly_trends_count = monthly_trends.count()
print(f"✅ Monthly trends: {monthly_trends_count} records")

In [0]:
# Performance Optimization
print("⚡ OPTIMIZING GOLD TABLES")
print("=" * 30)

optimization_results = []

# Optimize all Gold tables
gold_tables = [
    ("sales_dashboard", ["order_year", "order_month"]),
    ("customer_segments", ["customer_tier"]),
    ("product_performance", ["category"]),
    ("executive_kpis", []),
    ("monthly_trends", ["order_year", "order_month"])
]

for table_name, z_order_cols in gold_tables:
    print(f"🔧 Optimizing ecommerce_gold.{table_name}...")
    
    # Basic optimization
    spark.sql(f"OPTIMIZE ecommerce_gold.{table_name}")
    
    # Z-ordering if columns specified
    if z_order_cols:
        z_order_cols_str = ", ".join(z_order_cols)
        spark.sql(f"OPTIMIZE ecommerce_gold.{table_name} ZORDER BY ({z_order_cols_str})")
        optimization_results.append(f"{table_name}: Z-ordered by {z_order_cols_str}")
    else:
        optimization_results.append(f"{table_name}: Optimized")

print("✅ All Gold tables optimized")

In [0]:
# Calculate final metrics and set task values
gold_processing_timestamp = str(datetime.now())

# Calculate total Gold records
total_gold_records = (sales_dashboard_count + customer_segments_count + 
                     product_performance_count + executive_kpis_count + monthly_trends_count)

# Get final revenue from executive KPIs
final_revenue_overall = executive_kpis.filter(F.col("period") == "Overall").select("total_revenue").collect()[0][0]
final_customers_overall = executive_kpis.filter(F.col("period") == "Overall").select("active_customers").collect()[0][0]
final_avg_order_value = executive_kpis.filter(F.col("period") == "Overall").select("avg_order_value").collect()[0][0]
completion_rate = executive_kpis.filter(F.col("period") == "Overall").select("completion_rate_pct").collect()[0][0]

# Calculate pipeline efficiency metrics
try:
    end_to_end_data_retention = (final_customers_overall / original_active_customers * 100) if original_active_customers > 0 else 100
    revenue_accuracy = (1 - abs(final_revenue_overall - original_total_revenue) / original_total_revenue) * 100 if original_total_revenue > 0 else 100
except:
    end_to_end_data_retention = 100
    revenue_accuracy = 100

# Set comprehensive task values for monitoring and reporting
dbutils.jobs.taskValues.set(key="gold_total_records", value=total_gold_records)
dbutils.jobs.taskValues.set(key="gold_sales_dashboard_count", value=sales_dashboard_count)
dbutils.jobs.taskValues.set(key="gold_customer_segments_count", value=customer_segments_count)
dbutils.jobs.taskValues.set(key="gold_product_performance_count", value=product_performance_count)
dbutils.jobs.taskValues.set(key="gold_executive_kpis_count", value=executive_kpis_count)
dbutils.jobs.taskValues.set(key="gold_monthly_trends_count", value=monthly_trends_count)

# Business metrics
dbutils.jobs.taskValues.set(key="final_total_revenue", value=float(final_revenue_overall))
dbutils.jobs.taskValues.set(key="final_active_customers", value=int(final_customers_overall))
dbutils.jobs.taskValues.set(key="final_avg_order_value", value=float(final_avg_order_value))
dbutils.jobs.taskValues.set(key="order_completion_rate", value=float(completion_rate))

# Pipeline quality metrics
dbutils.jobs.taskValues.set(key="end_to_end_data_retention_pct", value=float(end_to_end_data_retention))
dbutils.jobs.taskValues.set(key="revenue_accuracy_pct", value=float(revenue_accuracy))
dbutils.jobs.taskValues.set(key="gold_processing_timestamp", value=gold_processing_timestamp)
dbutils.jobs.taskValues.set(key="gold_container_path", value=f"abfss://gold@{STORAGE_ACCOUNT}.dfs.core.windows.net/")

# Pipeline summary
dbutils.jobs.taskValues.set(key="pipeline_status", value="SUCCESS")
dbutils.jobs.taskValues.set(key="optimization_applied", value=True)
dbutils.jobs.taskValues.set(key="tables_optimized", value=len(gold_tables))

In [0]:
# Display comprehensive results
print("📊 GOLD LAYER BUSINESS ANALYTICS")
print("=" * 50)

# Show executive KPIs
print("📈 Executive KPIs:")
executive_kpis.show(truncate=False)

print("\n🎯 Top Customer Segments by Revenue:")
spark.sql("""
SELECT customer_tier, recency_status, customer_count, total_revenue, revenue_per_customer
FROM ecommerce_gold.customer_segments 
ORDER BY total_revenue DESC 
LIMIT 5
""").show()

print("\n📦 Top Product Categories by Performance:")
spark.sql("""
SELECT category, category_revenue, units_sold, profit_margin_pct, avg_discount_percent
FROM ecommerce_gold.product_performance 
ORDER BY category_revenue DESC
""").show()

print("\n📅 Recent Monthly Performance:")
spark.sql("""
SELECT month_year, orders, revenue, customers, avg_order_value, weekend_order_pct
FROM ecommerce_gold.monthly_trends 
ORDER BY order_year DESC, order_month DESC
LIMIT 6
""").show()

# COMMAND ----------

# Final Pipeline Summary
print("🏆 PIPELINE EXECUTION SUMMARY")
print("=" * 50)
print(f"📁 Gold Storage: abfss://gold@{STORAGE_ACCOUNT}.dfs.core.windows.net/")
print(f"📊 Gold Records Created: {total_gold_records:,}")
print(f"💰 Final Revenue: ${final_revenue_overall:,.2f}")
print(f"👥 Active Customers: {final_customers_overall:,}")
print(f"📈 Avg Order Value: ${final_avg_order_value:.2f}")
print(f"✅ Order Completion Rate: {completion_rate:.1f}%")
print(f"🎯 End-to-End Data Retention: {end_to_end_data_retention:.1f}%")
print(f"💯 Revenue Accuracy: {revenue_accuracy:.1f}%")
print(f"⚡ Tables Optimized: {len(gold_tables)}")

print(f"\n🕐 Processing Timeline:")
try:
    generation_time = dbutils.jobs.taskValues.get(taskKey="data_generation", key="data_generation_timestamp", debugValue="N/A")
    bronze_time = dbutils.jobs.taskValues.get(taskKey="bronze_ingestion", key="bronze_ingestion_timestamp", debugValue="N/A")
    silver_time = dbutils.jobs.taskValues.get(taskKey="silver_processing", key="silver_processing_timestamp", debugValue="N/A")
    print(f"   Data Generation: {generation_time}")
    print(f"   Bronze Ingestion: {bronze_time}")
    print(f"   Silver Processing: {silver_time}")
    print(f"   Gold Analytics: {gold_processing_timestamp}")
except:
    print("   Individual timestamps available in job run details")

print("\n✅ Gold layer analytics complete!")
print("📁 Data stored in Azure Gold container")
print("📋 Comprehensive task values set for monitoring")
print("🎯 Ready for dashboard consumption!")
print("🔜 Next: Query data using 05_dashboard_queries.sql")